In [1]:
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
sys.path.append('../src/methods')
sys.path.append("src")
sys.path.append("../")

In [2]:
from src.methods.analytic.exact import AnalyticalMathodOld

In [3]:
method = AnalyticalMathodOld()


In [4]:
import pandas as pd

In [14]:
main_dir = "../data/generated/perlin_ceramic_test/"
df1 = pd.read_csv(main_dir + "parameters.csv")
df2 = pd.read_csv(main_dir + "parameters2.csv")
df3 = pd.read_csv(main_dir + "parameters3.csv")

df = pd.concat([df1, df2, df3])
df.to_csv(main_dir + "parameters.csv", index=False)

In [16]:
df = pd.read_csv(main_dir + "parameters.csv")
df

,width,height,epsilon,ring_center_width,ring_center_height,min_brightness,max_brightness,filename,grid_size,clip_limit,persistance
0,640,480,0.000,326,239,82,196,perlin_00000.png,"(2, 2)",1.343294,0.779584
1,640,480,0.000,316,236,84,199,perlin_00001.png,"(4, 4)",1.686432,0.749954
2,640,480,0.000,319,243,87,205,perlin_00002.png,"(16, 16)",1.398639,0.855374
3,640,480,0.000,319,240,80,191,perlin_00003.png,"(4, 4)",1.462212,0.770618
4,640,480,0.000,321,237,83,200,perlin_00004.png,"(8, 8)",1.795448,0.823945
...,...,...,...,...,...,...,...,...,...,...,...
14995,640,480,0.999,321,237,93,197,perlin_14995.png,"(16, 16)",1.268804,0.814718
14996,640,480,0.999,319,235,97,197,perlin_14996.png,"(4, 4)",1.412112,0.779488
14997,640,480,0.999,317,236,89,209,perlin_14997.png,"(8, 8)",1.564023,0.792188
14998,640,480,0.999,324,237,100,206,perlin_14998.png,"(8, 8)",1.055617,0.786416


In [10]:
# read csv with steel images
all_steel_df = pd.read_csv("../data/generated/fourier/steel/parameters.csv", index_col=False)
avg_steel_df = pd.read_csv("../data/generated/fourier/single/parameters.csv", index_col=False)

# add prefix "all/steel/" to the filename column
all_steel_df['filename'] = "fourier/steel/" + all_steel_df['filename'].astype(str)
# the same for the average dataframe
avg_steel_df['filename'] = "fourier/single/" + avg_steel_df['filename'].astype(str)

main_dir = "../data/generated/"

all_steel_train, all_steel_test = train_test_split(all_steel_df, test_size=0.4, random_state=12, shuffle=True, stratify=all_steel_df['epsilon'])
all_steel_test, all_steel_valid = train_test_split(all_steel_test, test_size=1000, random_state=12, shuffle=True, stratify=all_steel_test['epsilon'])

avg_steel_train, avg_steel_test = train_test_split(avg_steel_df, test_size=0.4, random_state=12, shuffle=True, stratify=avg_steel_df['epsilon'])
avg_steel_test, avg_steel_valid = train_test_split(avg_steel_test, test_size=1000, random_state=12, shuffle=True, stratify=avg_steel_test['epsilon'])

In [17]:
# avg_steel_test
# iterate over the dataframe, read the image into numpy array using cv2 and run method.calculate_epsilon()
# save the result to the list. First image need to be loaded, function takes numpy array as input

# main_dir = "../data/raw/ceramic/1channel/"

# read the filenames from main_dir
import os
png_filenames = df['filename'].tolist()

results = []
from tqdm import tqdm
import cv2
for i in tqdm(png_filenames):
    img = cv2.imread(main_dir + i, cv2.IMREAD_GRAYSCALE)
    epsilon = method.calculate_epsilon(img)
    results.append(epsilon[0])


100%|██████████| 15000/15000 [03:14<00:00, 77.22it/s]


In [8]:
df = pd.DataFrame({'filename': png_filenames, 'epsilon': results})

df.to_csv("../data/results/exact_method/exact_method-raw_ceramic_predictions.csv", index=False)

In [11]:
def cyclic_MAE_test(output, target):
    return min(abs(output-target), min(abs(output-target+1), abs(output-target-1)))

In [12]:
def save_results(outputs, targets, filename: str):
    differences = []
    for out, tar in zip(outputs, targets):
        differences.append(cyclic_MAE_test(out, tar))
    results_df = pd.DataFrame(list(zip(targets, outputs, differences)),
                  columns =['epsilon', 'predicted', 'loss'])
    results_df.to_csv(filename, index=False)
    return results_df

In [18]:
results = save_results(results, df['epsilon'], "exact_method-perlin_average.csv")
results['loss'].mean()

0.04836181656229298